# Validating Streamlines

## Setup

### Imports

In [4]:
import os
from typing import List, Dict, Tuple, Any
import time

import numpy as np
from dotenv import load_dotenv
import nrrd
import h5py
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap

from src.core.utils import fs
from src.flatmap.utils.bridge import get_boundaries, get_isocortex_3d_projector, get_average_template_projection
from src.flatmap.utils.read_ni_output import get_marker_points_from_xml
from src.flatmap.ccf_streamlines.morphology import transform_coordinates_to_volume
from src.flatmap.utils.nb_utils import get_projection_images_from_dict, setup_main_plot, plot_boundaries
from src.flatmap.utils.nb_utils import expand_label_image, dilate_image, dilate_binary_image

from src.flatmap import external_data_registry as data_registry

In [2]:
if load_dotenv() is False:
    print("Failed to load .env file")

ALLEN_DATA_DIR = os.getenv("ALLEN_ATLAS_FILES")

if ALLEN_DATA_DIR is None:
    print("Failed to load ALLEN_ATLAS_FILES from .env file")

In [3]:
CORTEX_MASK_FILE = os.path.join(ALLEN_DATA_DIR, "isocortex_mask_10.nrrd")
ANNOTATION_VOLUME_FILE = os.path.join(ALLEN_DATA_DIR, "annotation_10.nrrd")

cortex_mask_volume, _ = nrrd.read(CORTEX_MASK_FILE)
annotation_volume, _ = nrrd.read(ANNOTATION_VOLUME_FILE)

In [ ]:
def _load_and_sort_paths(surface_paths_file) -> Tuple[np.ndarray, np.ndarray]:
    with h5py.File(surface_paths_file, "r") as path_f:
        paths = path_f["paths"][:]
        volume_lookup_dset = path_f["volume lookup flat"]
        volume_shape = tuple(volume_lookup_dset.attrs["original shape"])

        # Select and order paths to match the projection.
        # The view_lookup array contains the indices of the 2D view in the first
        # column and indices of the (flattened) 3D volume in the second.
        # We find the indices of the paths by going to the appropriate voxels
        # in volume_lookup.
        print("Sorting paths to match view lookup")

        view_sorter = np.argsort(view_lookup[:, 1])
        view_unsorter = np.argsort(view_sorter)

        # pull chunks from volume lookup to reduce memory usage
        chunk_size = 1000
        sorted_lookup = self.view_lookup[view_sorter, 1]
        path_ind = np.zeros_like(sorted_lookup)
        print("loading path information")
        for i in tqdm(range(0, sorted_lookup.shape[0], chunk_size)):
            # track duplicate info
            vals, counts = np.unique(sorted_lookup[i:i + chunk_size], return_counts=True)
            arr = volume_lookup_dset[vals]
            path_ind[i:i + chunk_size] = np.repeat(arr, counts)

In [ ]:
ISOCORTEX_LAYER_KEYS = [
    'Isocortex layer 1',
    'Isocortex layer 2/3',  # hilariously, this goes into a group in the h5 file
    'Isocortex layer 4',
    'Isocortex layer 5',
    'Isocortex layer 6a',
    'Isocortex layer 6b'
]
path_ordering_file = data_registry.get_streamline_path("surface_paths_10_v3")
streamline_layer_thickness_file = data_registry.get_isocortex_metric_path("cortical_layers_10_v2")
path_layer_thickness = {}
with h5py.File(streamline_layer_thickness_file, "r") as f:
    for k in ISOCORTEX_LAYER_KEYS:
        path_layer_thickness[k] = f[k][:]

        # Select and order paths to match the projection.
        path_layer_thickness[k] = path_layer_thickness[k][path_ordering, :]